# Test models on the halogenases dataset

In [ ]:
# define the directory where the data is
data_path = "/home/joao/Desktop/required_data_ec_number_paper/"

## Create useful functions to post process the predictions

In [ ]:
import re
import pandas as pd


def get_unique_labels_by_level(dataset, level):
    final_dataset_test = dataset.copy()
    final_dataset_test = final_dataset_test.loc[:,level]
    final_dataset_test.fillna("0", inplace=True)
    values = pd.Series(final_dataset_test.values.reshape(-1)).str.split(";")
    list_of_unique_labels = np.unique(values.explode()).tolist()
    if "0" in list_of_unique_labels:
        list_of_unique_labels.remove("0")
    list_of_unique_labels_dict = dict(zip(list_of_unique_labels, range(len(list_of_unique_labels))))
    return list_of_unique_labels_dict

def get_final_labels(dataset, all_levels=False):

    if all_levels:
        unique_EC1 = get_unique_labels_by_level(dataset, "EC1")
        unique_EC2 = get_unique_labels_by_level(dataset, "EC2")
        unique_EC3 = get_unique_labels_by_level(dataset, "EC3")
        
        array_EC1 = np.zeros((len(dataset), len(unique_EC1)))
        array_EC2 = np.zeros((len(dataset), len(unique_EC2)))
        array_EC3 = np.zeros((len(dataset), len(unique_EC3)))

    unique_EC4 = get_unique_labels_by_level(dataset, "EC4")

    array_EC4 = np.zeros((len(dataset), len(unique_EC4)))
        
    dataset.fillna("0", inplace=True)

    if all_levels:
        for i, row in dataset.iterrows():
            for ec in ["EC1", "EC2", "EC3", "EC4"]:
                for EC in row[ec].split(";"):
                    if EC != "0":
                        if ec == "EC1":
                            array_EC1[i, unique_EC1[EC]] = 1
                        elif ec == "EC2":
                            array_EC2[i, unique_EC2[EC]] = 1
                        elif ec == "EC3":
                            array_EC3[i, unique_EC3[EC]] = 1
                        elif ec == "EC4":
                            array_EC4[i, unique_EC4[EC]] = 1
    else:
        for i, row in dataset.iterrows():
            for EC in row["EC4"].split(";"):
                if EC != "0":
                    array_EC4[i, unique_EC4[EC]] = 1
    if all_levels:
        array_EC1 = pd.DataFrame(array_EC1, columns=unique_EC1.keys())
        array_EC2 = pd.DataFrame(array_EC2, columns=unique_EC2.keys())
        array_EC3 = pd.DataFrame(array_EC3, columns=unique_EC3.keys())
    array_EC4 = pd.DataFrame(array_EC4, columns=unique_EC4.keys())

    if all_levels:
        dataset = pd.concat((dataset, array_EC1, array_EC2, array_EC3, array_EC4), axis=1)
    else:
        dataset = pd.concat((dataset, array_EC4), axis=1)
    return dataset


def get_ec_from_regex_match(match):
    if match is not None:
        EC = match.group()
        if EC is not None:
            return EC
    return None

def get_labels_based_on_list(dataset, labels, all_levels=True):
    array = np.zeros((len(dataset), len(labels)))
    labels_dataframe = pd.DataFrame(array, columns=labels)
    dataset.fillna("0", inplace=True)
    for i, row in dataset.iterrows():
        for label in row["EC4"].split(";"):
            if label != "0" and label in labels:
                labels_dataframe.at[i, label] = 1
        
        if all_levels:
            for ec in ["EC1", "EC2", "EC3"]:
                for label in row[ec].split(";"):
                    if label != "0" and label in labels:
                        labels_dataframe.at[i, label] = 1

    return pd.concat((dataset, labels_dataframe), axis=1)

def divide_labels_by_EC_level(final_dataset, ec_label):
    EC1_lst = []
    EC2_lst = []
    EC3_lst = []
    EC4_lst = []


    for _, row in final_dataset.iterrows():
        ECs = row[ec_label]
        ECs = ECs.split(";")
        # get the first 3 ECs with regular expression
        EC3 = []
        EC2 = []
        EC1 = []
        EC4 = []
        for EC in ECs:
            new_EC = re.search(r"^\d+.\d+.\d+.n*\d+", EC)
            new_EC = get_ec_from_regex_match(new_EC)
            if isinstance(new_EC, str):
                if new_EC not in EC4:
                    EC4.append(new_EC)

            new_EC = re.search(r"^\d+.\d+.\d+", EC)
            new_EC = get_ec_from_regex_match(new_EC)
            if isinstance(new_EC, str):
                if new_EC not in EC3:
                    EC3.append(new_EC)

            new_EC = re.search(r"^\d+.\d+", EC)
            new_EC = get_ec_from_regex_match(new_EC)
            if isinstance(new_EC, str):
                if new_EC not in EC2:
                    EC2.append(new_EC)

            new_EC = re.search(r"^\d+", EC)
            new_EC = get_ec_from_regex_match(new_EC)
            if isinstance(new_EC, str):
                if new_EC not in EC1:
                    EC1.append(new_EC)

        if len(EC4) == 0:
            EC4_lst.append(np.NaN)
        else:
            EC4_lst.append(";".join(EC4))
        if len(EC3) == 0:
            EC3_lst.append(np.NaN)
        else:
            EC3_lst.append(";".join(EC3))
        if len(EC2) == 0:
            EC2_lst.append(np.NaN)
        else:
            EC2_lst.append(";".join(EC2))
        if len(EC1) == 0:
            EC1_lst.append(np.NaN)
        else:
            EC1_lst.append(";".join(EC1))

    assert None not in EC1_lst
    assert None not in EC2_lst
    assert None not in EC3_lst
    assert None not in EC4_lst

    assert len(EC1_lst) == len(final_dataset)
    assert len(EC2_lst) == len(final_dataset)
    assert len(EC3_lst) == len(final_dataset)
    assert len(EC4_lst) == len(final_dataset)

    final_dataset["EC1"] = EC1_lst
    final_dataset["EC2"] = EC2_lst
    final_dataset["EC3"] = EC3_lst
    final_dataset["EC4"] = EC4_lst

    assert final_dataset["EC1"].isnull().sum() == 0
    print("EC1 is not null")

    return final_dataset

In [ ]:
from sklearn.metrics import accuracy_score

def get_ec_levels(labels):
    level_1 = []
    level_2 = []
    level_3 = []
    level_4 = []
    for i, label in enumerate(labels):
        if re.match(r"^\d+.\d+.\d+.n*\d+$", label):
            level_4.append(i)
        elif re.match(r"^\d+.\d+.\d+$", label):
            level_3.append(i)
        elif re.match(r"^\d+.\d+$", label):
            level_2.append(i)
        elif re.match(r"^\d+$", label):
            level_1.append(i)
    return level_1, level_2, level_3, level_4

def get_metrics(y_true, predictions, labels, labels_to_remove, model_name):

    y_true_ = np.delete(y_true, labels_to_remove, axis=1)
    predictions_ = np.delete(predictions, labels_to_remove, axis=1)
    labels_ = np.delete(labels, labels_to_remove)
    level_1, level_2, level_3, level_4 = get_ec_levels(labels_)
    print("level 1", len(level_1))
    print("level 2", len(level_2))
    print("level 3", len(level_3))
    print("level 4", len(level_4))
    
    metrics = {}
    # metrics["accuracy overall"] = accuracy_score(y_true, predictions)
    metrics["accuracy level 1"] = accuracy_score(y_true_[:, level_1], predictions_[:, level_1])
    metrics["accuracy level 2"] = accuracy_score(y_true_[:, level_2], predictions_[:, level_2])
    metrics["accuracy level 3"] = accuracy_score(y_true_[:, level_3], predictions_[:, level_3])
    metrics["accuracy level 4"] = accuracy_score(y_true_[:, level_4], predictions_[:, level_4])

    return pd.DataFrame(metrics, index=[model_name])

def get_models_predictions(model, dataset, labels, labels_to_remove, model_name):
    predictions = model.predict(dataset)
    y_true = dataset.y

    return get_metrics(y_true, predictions, labels, labels_to_remove, model_name)

def convert_predictions_into_format(predictions, labels_names, new_labels):
    new_predictions = np.zeros((len(predictions), len(new_labels)))
    labels_names = np.array(labels_names)
    new_labels = np.array(new_labels)
    for i, prediction in enumerate(predictions):
        indexes = np.where(prediction == 1)
        result = labels_names[indexes]
        for res in result:
            potential_result = new_labels[new_labels == res]
            if potential_result.size > 0:
                new_predictions[i, new_labels == res] = 1
    return new_predictions

## Read the dataset

In [ ]:
dataset = pd.read_csv(f'{data_path}/data/halogenase.csv', sep="\t")

In [101]:
from plants_sm.data_structures.dataset.single_input_dataset import SingleInputDataset

halogenases_dataset = SingleInputDataset.from_csv(f'{data_path}/data/halogenase.csv',
                                            instances_ids_field="Entry", representation_field="Sequence", sep="\t",
                                            nrows=36)

In [103]:
h_dataset = pd.read_csv(f'{data_path}/data/halogenase.csv', sep="\t", nrows=36)
h_dataset = divide_labels_by_EC_level(h_dataset, "EC number")
h_dataset_ = get_final_labels(h_dataset, all_levels=True)

EC1 is not null


In [104]:
from plants_sm.data_structures.dataset.single_input_dataset import SingleInputDataset

labels_names = SingleInputDataset.from_csv(f'{data_path}/data/merged_dataset.csv',
                                           instances_ids_field="accession", representation_field="sequence",
                                           labels_field=slice(8, -1), nrows=2).labels_names

In [105]:
len(labels_names)

5743

# ESM2 3B - get predictions

In [106]:
from plants_sm.models.fc.fc import DNN
from plants_sm.models.pytorch_model import PyTorchModel
from torch import nn
import torch

model = PyTorchModel(model = DNN(2560, [2560], 5743, batch_norm=True), loss_function=nn.BCELoss())
model.model.load_state_dict(
    torch.load(f'{data_path}/models/DNN_esm2_t36_3B_UR50D_optimization_set_2_all_data/pytorch_model_weights.pt', map_location=torch.device('cpu')))


<All keys matched successfully>

In [107]:
import os
from plants_sm.data_standardization.proteins.standardization import ProteinStandardizer
from plants_sm.data_standardization.truncation import Truncator
from plants_sm.featurization.proteins.bio_embeddings.esm import ESMEncoder

encoding = "esm2_t36_3B_UR50D"

if not os.path.exists(f'{data_path}/features/test_halogenases_esm2_3b'):
    transformers = [ProteinStandardizer(), Truncator(max_length=884), ESMEncoder(esm_function=encoding, batch_size=1, num_gpus=4)]
    for transformer in transformers:
        halogenases_dataset = transformer.fit_transform(halogenases_dataset)
    halogenases_dataset.save_features(f'{data_path}/features/test_halogenases_esm2_3b')
else:
    halogenases_dataset.load_features(f'{data_path}/features/test_halogenases_esm2_3b')

In [108]:
predictions = model.predict(halogenases_dataset)
new_predictions_esm2_3b = convert_predictions_into_format(predictions, labels_names, h_dataset_.iloc[:, 7:].columns)
y_true = h_dataset_.iloc[:, 7:].values
metrics_evaluation = get_metrics(y_true, new_predictions_esm2_3b, h_dataset_.iloc[:, 7:].columns, [], "DNN ESM2 3B")
metrics_evaluation

level 1 3
level 2 4
level 3 6
level 4 13


,accuracy level 1,accuracy level 2,accuracy level 3,accuracy level 4
DNN ESM2 3B,0.972222,0.972222,0.555556,0.305556


In [109]:
len(labels_names)

5743

# ESM1b - get predictions

In [110]:
from plants_sm.data_structures.dataset.single_input_dataset import SingleInputDataset

halogenases_dataset = SingleInputDataset.from_csv(f'{data_path}/data/halogenase.csv',
                                            instances_ids_field="Entry", representation_field="Sequence", sep="\t", nrows=36)

In [111]:
model = PyTorchModel(model = DNN(1280, [2560, 5120], 5743, batch_norm=True), loss_function=nn.BCELoss())
model.model.load_state_dict(
    torch.load(f'{data_path}/DNN_esm1b_t33_650M_UR50S_optimization_set_4_all_data/pytorch_model_weights.pt', map_location=torch.device('cpu')))


<All keys matched successfully>

In [112]:
import os
from plants_sm.data_standardization.proteins.standardization import ProteinStandardizer
from plants_sm.data_standardization.truncation import Truncator
from plants_sm.featurization.proteins.bio_embeddings.esm import ESMEncoder

encoding = "esm1b_t33_650M_UR50S"

if not os.path.exists(f'{data_path}/features/test_halogenases_esm1b'):
    transformers = [ProteinStandardizer(), Truncator(max_length=884), ESMEncoder(esm_function=encoding, batch_size=1, num_gpus=4)]
    for transformer in transformers:
        halogenases_dataset = transformer.fit_transform(halogenases_dataset)
    halogenases_dataset.save_features(f'{data_path}/features/test_halogenases_esm1b')
else:
    halogenases_dataset.load_features(f'{data_path}/features/test_halogenases_esm1b')

In [113]:
predictions = model.predict(halogenases_dataset)
new_predictions_esm1b = convert_predictions_into_format(predictions, labels_names, h_dataset_.iloc[:, 7:].columns)
y_true = h_dataset_.iloc[:, 7:].values
metrics_evaluation = pd.concat((metrics_evaluation, get_metrics(y_true, new_predictions_esm1b, h_dataset_.iloc[:, 7:].columns, [], "DNN ESM1b")))
metrics_evaluation

level 1 3
level 2 4
level 3 6
level 4 13


,accuracy level 1,accuracy level 2,accuracy level 3,accuracy level 4
DNN ESM2 3B,0.972222,0.972222,0.555556,0.305556
DNN ESM1b,0.972222,0.972222,0.555556,0.333333


# ProtBERT - get predictions

In [114]:
from plants_sm.data_structures.dataset.single_input_dataset import SingleInputDataset

halogenases_dataset = SingleInputDataset.from_csv(f'{data_path}/data/halogenase.csv',
                                            instances_ids_field="Entry", representation_field="Sequence", sep="\t", nrows=36)

In [115]:
from plants_sm.models.pytorch_model import PyTorchModel

model = PyTorchModel(model = DNN(1024, [2560], 5743, batch_norm=True), loss_function=nn.BCELoss())
model.model.load_state_dict(
    torch.load(f'{data_path}/models/DNN_prot_bert_vectors_optimization_set_2_all_data/pytorch_model_weights.pt', map_location=torch.device('cpu')))


import os
from plants_sm.featurization.proteins.bio_embeddings.prot_bert import ProtBert
from plants_sm.data_standardization.proteins.standardization import ProteinStandardizer
from plants_sm.data_standardization.truncation import Truncator

if not os.path.exists(f'{data_path}/features/test_halogenases_prot_bert'):
    transformers = [ProteinStandardizer(), Truncator(max_length=884), ProtBert(device="cuda:0")]
    for transformer in transformers:
        halogenases_dataset = transformer.fit_transform(halogenases_dataset)
    halogenases_dataset.save_features(f'{data_path}/features/test_halogenases_prot_bert')
else:
    halogenases_dataset.load_features(f'{data_path}/features/test_halogenases_prot_bert')

In [116]:
predictions = model.predict(halogenases_dataset)
new_predictions_protein_bert = convert_predictions_into_format(predictions, labels_names, h_dataset_.iloc[:, 7:].columns)
y_true = h_dataset_.iloc[:, 7:].values
metrics_evaluation = pd.concat((metrics_evaluation, get_metrics(y_true, new_predictions_protein_bert, h_dataset_.iloc[:, 7:].columns, [], "DNN ProtBERT")))
metrics_evaluation

level 1 3
level 2 4
level 3 6
level 4 13


,accuracy level 1,accuracy level 2,accuracy level 3,accuracy level 4
DNN ESM2 3B,0.972222,0.972222,0.555556,0.305556
DNN ESM1b,0.972222,0.972222,0.555556,0.333333
DNN ProtBERT,0.861111,0.972222,0.583333,0.250000


In [118]:
metrics_evaluation.to_csv("metrics_evaluation_halogenases.csv")

# Test ensemble models on the halogenases dataset

In [119]:
# create an ensemble of the predictions
# make a voting classifier for the 3 models
import numpy as np

def determine_ensemble_predictions(threshold=3, *model_predictions):
    model_predictions = list(model_predictions)

    for i, model_prediction in enumerate(model_predictions):
        model_predictions[i] = np.array(model_prediction)


    predictions_voting = np.zeros_like(model_predictions[0])

    for i in range(model_predictions[0].shape[0]):
        # Combine conditions into a single array and sum along the second axis
        combined_conditions = np.sum(np.array([model_predictions[j][i] for j in range(len(model_predictions))]), axis=0)

        # Apply the threshold condition
        predictions_voting[i] = (combined_conditions >= threshold).astype(int)

    # If you want to ensure the resulting array is of integer type
    predictions_voting = predictions_voting.astype(int)
    return predictions_voting

In [120]:
predictions_ensemble = determine_ensemble_predictions(2, new_predictions_esm1b, new_predictions_protein_bert, new_predictions_esm2_3b)
metrics_evaluation = pd.concat((metrics_evaluation, get_metrics(y_true, predictions_ensemble,  h_dataset_.iloc[:, 7:].columns, [], "Models Ensemble")))
metrics_evaluation

level 1 3
level 2 4
level 3 6
level 4 13


,accuracy level 1,accuracy level 2,accuracy level 3,accuracy level 4
DNN ESM2 3B,0.972222,0.972222,0.555556,0.305556
DNN ESM1b,0.972222,0.972222,0.555556,0.333333
DNN ProtBERT,0.861111,0.972222,0.583333,0.250000
Models Ensemble,0.972222,0.972222,0.555556,0.333333


In [121]:
blast_results = pd.read_csv(f'{data_path}/halogenase_blast_results.csv')
blast_results.drop_duplicates(subset=["qseqid"], inplace=True)
# Create a new column with the custom order as a categorical type
blast_results['CustomOrder'] = pd.Categorical(blast_results['qseqid'], categories=h_dataset.Entry, ordered=True)
blast_results.sort_values('CustomOrder', inplace=True)
blast_results.drop(columns=["CustomOrder"], inplace=True)

In [122]:
blast_results["EC1"] = blast_results["EC1"].astype(str)
blast_results["EC2"] = blast_results["EC2"].astype(str)
blast_results["EC3"] = blast_results["EC3"].astype(str)
blast_results["EC4"] = blast_results["EC4"].astype(str)

In [123]:
blast_results.fillna("0", inplace=True)
blast_results_ = get_final_labels(blast_results, all_levels=True)
blast_results_.drop(columns=["nan"], inplace=True)
labels_names = [ec_number.replace(".0", "") for ec_number in blast_results_.columns[18:].tolist()]
labels_names

['1',
 '2',
 '1.11',
 '1.14',
 '1.21',
 '2.5',
 '1.11.1',
 '1.14.11',
 '1.14.13',
 '1.14.14',
 '1.14.19',
 '1.14.20',
 '1.21.3',
 '2.5.1',
 '1.11.1.10',
 '1.14.11.16',
 '1.14.13.20',
 '1.14.19.56',
 '1.14.19.9',
 '1.14.20.15',
 '1.21.3.1',
 '2.5.1.63',
 '2.5.1.94']

In [124]:
blast_predictions = np.array(blast_results_.iloc[:36, 18:])
new_blast_predictions = convert_predictions_into_format(blast_predictions, labels_names, h_dataset_.iloc[:, 7:].columns)
predictions_ensemble = determine_ensemble_predictions(2, new_predictions_esm1b, new_predictions_protein_bert, new_predictions_esm2_3b, new_blast_predictions)
metrics_evaluation = pd.concat((metrics_evaluation, get_metrics(y_true, predictions_ensemble, h_dataset_.iloc[:, 7:].columns, [], "Models Ensemble + BLASTp")))

level 1 3
level 2 4
level 3 6
level 4 13


In [125]:
metrics_evaluation = pd.concat((metrics_evaluation, get_metrics(y_true, new_blast_predictions, h_dataset_.iloc[:, 7:].columns, [], "BLASTp")))

level 1 3
level 2 4
level 3 6
level 4 13


In [126]:
metrics_evaluation

,accuracy level 1,accuracy level 2,accuracy level 3,accuracy level 4
DNN ESM2 3B,0.972222,0.972222,0.555556,0.305556
DNN ESM1b,0.972222,0.972222,0.555556,0.333333
DNN ProtBERT,0.861111,0.972222,0.583333,0.250000
Models Ensemble,0.972222,0.972222,0.555556,0.333333
Models Ensemble + BLASTp,0.972222,0.972222,0.555556,0.361111
BLASTp,0.861111,0.833333,0.472222,0.361111


In [127]:
metrics_evaluation.to_csv("metrics_evaluation_halogenases.csv")